In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
covid = pd.read_csv('covid_com_meso.csv')
covid = covid[:1000]

In [3]:
for i in range(len(covid)):
    covid['localCaso'].iloc[i] = covid['localCaso'].iloc[i].split(sep='; ')[0]

C:\Users\malu.maia\Anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


**Q1:** Média da idade dos indivíduos representados no dataset.

In [4]:
def Q1(dataset):
    return dataset['idadeCaso'].mean()

In [5]:
def delta_Q1():
    # Considering that the oldest person is 120 and the youngest 0, the sensibility is 120
    oldest = 120
    youngest = 0
    
    return abs(oldest - youngest)

**Q2:**  Número de exames positivos (atributo *resultadoFinalExame*).

In [6]:
def Q2(dataset):
    return dataset['resultadoFinalExame'].value_counts()['Positivo']

In [7]:
def delta_Q2():
    # Considering that the max number of positive exams is the Ceará's population
    # and the min is 0
    max_positive = 9240580
    min_positive = 0
    
    return abs(max_positive, min_positive)

**Q3:** Total de exames realizados por município (atributo *municipioCaso*).

In [8]:
def Q3(dataset):
    counties = np.unique(dataset['localCaso'])
    n_exams_county = {}
    for county in counties:
        n_exams = sum(dataset['localCaso'][dataset['localCaso'] == county].value_counts())
        n_exams_county[county] = n_exams
    
    return n_exams_county

In [9]:
def delta_Q3():
    # Considering that the max is the poulation of each county
    
    return

# Ruído de Laplace

In [10]:
def laplace_query(output, delta, eps):
    if isinstance(output, dict):
        eps = eps/len(output)
        cities = list(output.keys())
        counts = list(output.values())
        output_noise = counts + stats.laplace.rvs(loc=0, scale=delta/eps, size=len(output))
        
        return dict(zip(cities, output_noise))
    
    return output + stats.laplace.rvs(loc=0, scale=delta/eps)

In [11]:
epsilon = [0.1, 0.5, 1, 10]
queries = [Q1, Q2, Q3]

In [13]:
#all_results = []
#for q in queries:
#    outputs = {}
#    for eps in epsilon:
#        outputs[eps] = laplace_query()
#    all_results.append(outputs)